inspired from: https://youtu.be/cEgF0YknpZw

## Setup

In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0, 1, 2, 3' 

In [2]:
# !python -m pip install pyyaml==5.1
import sys, os, distutils.core
# !git clone 'https://github.com/facebookresearch/detectron2'
dist = distutils.core.run_setup("./detectron2/setup.py")
!python -m pip install {' '.join([f"'{x}'" for x in dist.install_requires])}
sys.path.insert(0, os.path.abspath('./detectron2'))

# !python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Ignoring dataclasses: markers 'python_version < "3.7"' don't match your environment


In [3]:
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243
torch:  2.1 ; cuda:  cu121
detectron2: 0.6


In [4]:
DEVICE = 'cuda:3'

In [5]:
from detectron2.utils.logger import setup_logger
setup_logger()
import numpy as np
import os, json, cv2, random
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

matplotlib data path: /opt/conda/envs/env_py311/lib/python3.11/site-packages/matplotlib/mpl-data
CONFIGDIR=/root/.config/matplotlib
interactive is False
platform is linux
CACHEDIR=/root/.cache/matplotlib
Using fontManager instance from /root/.cache/matplotlib/fontlist-v330.json


## Create configuration

In [6]:
dataset_train = 'my_dataset_train5'
dataset_val = 'my_dataset_val5'

In [7]:
from detectron2.data.datasets import register_coco_instances
register_coco_instances(
    dataset_train, 
    {}, 
    "./roboflow_datasets/xmm_om_artefacts_512-20-COCO-splits/train_1/skf_train_annotations.coco.json", 
    "./roboflow_datasets/xmm_om_artefacts_512-20-COCO-splits/train_1/")

register_coco_instances(
    dataset_val, 
    {}, 
    "./roboflow_datasets/xmm_om_artefacts_512-20-COCO-splits/valid_1/skf_valid_annotations.coco.json", 
    "./roboflow_datasets/xmm_om_artefacts_512-20-COCO-splits/valid_1/")

In [8]:
train_metadata = MetadataCatalog.get(dataset_train)
train_dataset_dicts = DatasetCatalog.get(dataset_train)

WARNING [04/09 17:00:16 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[04/09 17:00:16 d2.data.datasets.coco]: Loaded 484 images in COCO format from ./roboflow_datasets/xmm_om_artefacts_512-20-COCO-splits/train_1/skf_train_annotations.coco.json


In [9]:
val_metadata = MetadataCatalog.get(dataset_val)
val_dataset_dicts = DatasetCatalog.get(dataset_val)

WARNING [04/09 17:00:16 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[04/09 17:00:16 d2.data.datasets.coco]: Loaded 243 images in COCO format from ./roboflow_datasets/xmm_om_artefacts_512-20-COCO-splits/valid_1/skf_valid_annotations.coco.json


In [10]:
from matplotlib import pyplot as plt

In [11]:
# for d in random.sample(train_dataset_dicts, 2):
#     img = cv2.imread(d["file_name"])
#     visualizer = Visualizer(img[:, :, ::-1], metadata=train_metadata, scale=0.5)
#     vis = visualizer.draw_dataset_dict(d)
#     plt.imshow(vis.get_image()[:, :, ::-1])
#     plt.show()

In [12]:
import wandb
wandb.login()
wandb.init(project='ft_detectron2', sync_tensorboard=True)

Popen(['git', 'version'], cwd=/workspace/raid/OM_DeepLearning/XMM_OM_code_git, stdin=None, shell=False, universal_newlines=False)
Popen(['git', 'version'], cwd=/workspace/raid/OM_DeepLearning/XMM_OM_code_git, stdin=None, shell=False, universal_newlines=False)
Trying paths: ['/root/.docker/config.json', '/root/.dockercfg']
No config file found
[Tracing] Create new propagation context: {'trace_id': '661287c5267148d4b5054c361eef508d', 'span_id': '9a4d8176b9d041c3', 'parent_span_id': None, 'dynamic_sampling_context': None}
Starting new HTTP connection (1): localhost:8887
http://localhost:8887 "GET /api/sessions?token= HTTP/1.1" 403 40


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Starting new HTTPS connection (1): api.wandb.ai:443
https://api.wandb.ai:443 "POST /graphql HTTP/1.1" 200 None
https://api.wandb.ai:443 "POST /graphql HTTP/1.1" 200 None


wandb: Currently logged in as: iuliaelisa15. Use `wandb login --relogin` to force relogin
2024-04-09 17:00:18.554379: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Falling back to TensorFlow client; we recommended you install the Cloud TPU client directly with pip install cloud-tpu-client.
Creating converter from 7 to 5
Creating converter from 5 to 7
Creating converter from 7 to 5
Creating converter from 5 to 7


2024-04-09 17:00:19.425964: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Popen(['git', 'cat-file', '--batch-check'], cwd=/workspace/raid/OM_DeepLearning/XMM_OM_code_git, stdin=<valid stream>, shell=False, universal_newlines=False)


In [13]:
from detectron2.engine import DefaultTrainer

cfg = get_cfg()
cfg.OUTPUT_DIR = "./output_detectron2"
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = (dataset_train,)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml") 
cfg.SOLVER.IMS_PER_BATCH = 32 # BATCH_SIZE
cfg.SOLVER.BASE_LR = 0.0003  
cfg.SOLVER.MAX_ITER = 5000
cfg.SOLVER.STEPS = []        # do not decay learning rate
cfg.MODEL.PIXEL_MEAN = [38, 38, 38]
cfg.MODEL.PIXEL_STD = [1.0, 1.0, 1.0]
cfg.SOLVER.BASE_LR_END = 0.0
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512  
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 4 # number of classes, without the background!
cfg.SOLVER.AMP.ENABLED = True
cfg.SOLVER.WEIGHT_DECAY = 0.00005
cfg.MODEL.DEVICE = 'cuda:0'
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False) #Load a pretrained model if available (resume training) or start training from scratch if no pretrained model is available

IN GENERALIZED RCNN INIT
[04/09 17:00:41 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
   

Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (5, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (5,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 1024) in the checkpoint but (16, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (320,) in the checkpoint but (16,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.mask_head.predictor.weight' to the model due to incompatible shapes: (80, 256, 1, 1) in the checkpoint but (4, 256, 1, 1) 

## Train

In [14]:
trainer.train() 

[04/09 17:00:41 d2.engine.train_loop]: Starting training from iteration 0


/opt/conda/envs/env_py311/lib/python3.11/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3526.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
/opt/conda/envs/env_py311/lib/python3.11/site-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


[04/09 17:01:59 d2.utils.events]:  eta: 4:57:41  iter: 19  total_loss: 3.168  loss_cls: 1.498  loss_box_reg: 0.09642  loss_mask: 0.6925  loss_rpn_cls: 0.8262  loss_rpn_loc: 0.08431    time: 3.5860  last_time: 3.5161  data_time: 0.2723  last_data_time: 0.2370   lr: 5.9943e-06  max_mem: 10241M


2024-04-09 17:02:00.076834: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-09 17:02:00.922147: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


[04/09 17:03:12 d2.utils.events]:  eta: 4:59:08  iter: 39  total_loss: 2.74  loss_cls: 1.197  loss_box_reg: 0.1023  loss_mask: 0.69  loss_rpn_cls: 0.7074  loss_rpn_loc: 0.08591    time: 3.6294  last_time: 3.6891  data_time: 0.2400  last_data_time: 0.2411   lr: 1.1988e-05  max_mem: 10270M
[04/09 17:04:26 d2.utils.events]:  eta: 5:00:08  iter: 59  total_loss: 2.03  loss_cls: 0.7228  loss_box_reg: 0.1041  loss_mask: 0.6846  loss_rpn_cls: 0.3945  loss_rpn_loc: 0.08386    time: 3.6477  last_time: 3.6855  data_time: 0.2367  last_data_time: 0.2408   lr: 1.7982e-05  max_mem: 10310M
[04/09 17:05:39 d2.utils.events]:  eta: 4:58:55  iter: 79  total_loss: 1.476  loss_cls: 0.3859  loss_box_reg: 0.1017  loss_mask: 0.6777  loss_rpn_cls: 0.241  loss_rpn_loc: 0.07861    time: 3.6521  last_time: 3.8553  data_time: 0.2390  last_data_time: 0.2313   lr: 2.3976e-05  max_mem: 10335M
[04/09 17:06:53 d2.utils.events]:  eta: 4:58:00  iter: 99  total_loss: 1.298  loss_cls: 0.2449  loss_box_reg: 0.1123  loss_mask

In [ ]:
import yaml
# Save configuration
config_yaml_path = "./detr2_config.yaml"
with open(config_yaml_path, 'w') as file:
    yaml.dump(cfg, file)

# Inference using the trained model

In [ ]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  
predictor = DefaultPredictor(cfg)

In [ ]:
# S0655343836_L

img_fname = [train_dataset_dicts[i]['file_name'] for i in range(len(train_dataset_dicts)) \
             if train_dataset_dicts[i]['file_name'].split('/')[-1].startswith('S0412991401_U')]
img_fname

In [ ]:
len(val_dataset_dicts), len(train_dataset_dicts)

In [ ]:
train_dataset_dicts[0]['file_name'].split('/')[-1].startswith('S0018141301_M')

In [ ]:
from detectron2.utils.visualizer import ColorMode

# for d in val_dataset_dicts[:10]:    
im = cv2.imread(img_fname[0])
outputs = predictor(im)
instances = outputs["instances"]
print(d["file_name"].split("/")[-1])
if instances.has("pred_masks") and instances.has("pred_classes"):
    masks = instances.pred_masks.to("cpu").numpy()  # A binary mask of shape [num_instances, H, W]
    classes = instances.pred_classes.to("cpu").numpy()  # An array of class IDs for each instance
    scores = instances.scores.to("cpu").numpy()  # An array of scores for each instance
    boxes = instances.pred_boxes.tensor.to("cpu").numpy()  # An array of bounding boxes for each instance, shape [num_instances, 4]
    plt.imshow(im)
    plt.title(f'Detectron2 Masks')
    dataset_utils.show_masks(masks, plt.gca()) #, random_color=True)
    plt.show()
    plt.close()
        
    # v = Visualizer(im[:, :, ::-1],
    #                metadata=val_metadata,
    #                scale=0.5,
    #                instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels. This option is only available for segmentation models
    # )
    # out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    # plt.imshow(out.get_image()[:, :, ::-1])

In [16]:
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
evaluator = COCOEvaluator(dataset_val, output_dir="./output_detectron2")
val_loader = build_detection_test_loader(cfg,dataset_val)
print(inference_on_dataset(predictor.model, val_loader, evaluator))

[04/04 21:31:00 d2.evaluation.coco_evaluation]: Fast COCO eval is not built. Falling back to official COCO eval.
WARNING [04/04 21:31:00 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[04/04 21:31:00 d2.data.datasets.coco]: Loaded 243 images in COCO format from ./roboflow_datasets/xmm_om_artefacts_512-20-COCO-splits/valid_1/skf_valid_annotations.coco.json
[04/04 21:31:00 d2.data.build]: Distribution of instances among all 4 categories:
|  category  | #instances   |   category   | #instances   |  category  | #instances   |
|:----------:|:-------------|:------------:|:-------------|:----------:|:-------------|
| artefacts  | 0            | central-ring | 160          | smoke-ring | 382          |
| star-loop  | 444          |              |              |            |              |
|   total    | 986          |              |              |            |              |
[04/04 21:31:00 d2.data.dataset_mapper]: [DatasetMapp

In [18]:
# S0156960101_U

## Export new annotations to Roboflow

In [1]:
import glob
from roboflow import Roboflow
import os
import dataset 
from dataset import voc_annotate_and_Roboflow_export, dataset_utils
import cv2
import matplotlib.pyplot as plt
export_to_Roboflow = True

if export_to_Roboflow:
    # Initialize Roboflow client
    rf = Roboflow(api_key="EBeK30tpU3HW2VGGl0xa")
    upload_project = rf.workspace("iuliaelisa").project("xmm_om_artefacts_512") # error if the project doesn't exist
    
def export_image_det_to_Roboflow(input_dir, filename, masks, class_label, boxes):
        
        objects = []
        for i in range(len(masks)):
            mask_np = masks[i] # [H, W]
            polygon = voc_annotate_and_Roboflow_export.binary_image_to_polygon(mask_np)
            bbox = boxes[i]
            objects.append({
                'name': train_metadata.thing_classes[class_label[i]],
                'bbox': bbox,
                'segmentations': polygon[0]
            })
        if len(objects)>0:
            voc_annotate_and_Roboflow_export.create_annotation_SAM(
                filename=filename, 
                width=512, 
                height=512, 
                depth=3, 
                objects=objects, 
                offset= 1.3) # generating xml file for VOC format
            annotation_filename = filename.replace(".png", ".xml")
            upload_project.upload(input_dir+filename, annotation_filename, overwrite=True)
            os.remove(annotation_filename)
        else:
            upload_project.upload(input_dir+filename, overwrite=True)
    
# Directory path to the input images folder
input_images_directory = "../XMM_OM_dataset/zscaled_512_stretched/"

# Output directory where the segmented images will be saved
output_directory = "."  
import time
for image_filename in os.listdir(input_images_directory)[2035:2045]:
    if not image_filename.endswith('.json'):
        print(image_filename)
        image_path = os.path.join(input_images_directory, image_filename)
        new_im = cv2.imread(image_path)
        start_time = time.time()
        # Perform prediction on the new image
        outputs = predictor(new_im)  # Format is documented at https://detectron2.readthedocs.io/tutorials/models.html#model-output-format
        instances = outputs["instances"]
        print("Inference time per image:", time.time()-start_time)

        if instances.has("pred_masks") and instances.has("pred_classes"):
            masks = instances.pred_masks.to("cpu").numpy()  # A binary mask of shape [num_instances, H, W]
            classes = instances.pred_classes.to("cpu").numpy()  # An array of class IDs for each instance
            scores = instances.scores.to("cpu").numpy()  # An array of scores for each instance
            boxes = instances.pred_boxes.tensor.to("cpu").numpy()  # An array of bounding boxes for each instance, shape [num_instances, 4]
            # if export_to_Roboflow:
            #     export_image_det_to_Roboflow(input_images_directory, image_filename, masks, classes, boxes)
            plt.imshow(new_im)
            plt.title('Detectron2 Masks')
            dataset_utils.show_masks(masks, plt.gca()) #, random_color=True)
            # plt.savefig('./plots/detr2_masks.png')
            plt.show()
            plt.close()

print("Segmentation of all images completed.")

loading Roboflow workspace...
loading Roboflow project...
S0112231101_L.png


NameError: name 'predictor' is not defined